In [15]:
!pip install web3 py-solc-x eth-tester
!python -m solcx.install v0.8.0

<frozen runpy>:128: RuntimeWarning: 'solcx.install' found in sys.modules after import of package 'solcx', but prior to execution of 'solcx.install'; this may result in unpredictable behaviour


**WEEK 5 AND WEEK 6**

In [22]:
from solcx import compile_source, set_solc_version
from web3 import Web3
import json
import time

# Start Ganache i.e.in-memory blockchain
!npm install -g ganache
import subprocess
ganache_process = subprocess.Popen(["ganache", "--port", "8545", "--quiet"])


time.sleep(3)


w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
w3.eth.default_account = w3.eth.accounts[0]

print(f"Connected: {w3.is_connected()}")
print(f"Accounts: {w3.eth.accounts}")
print(f"Balance: {w3.from_wei(w3.eth.get_balance(w3.eth.accounts[0]), 'ether')} ETH")

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 336 packages in 15s
⠏
⠏6 packages are looking for funding
⠏  run `npm fund` for details
⠏Connected: True
Accounts: ['0xa33CF0c4DeDa373483cBB60f59f7AD7Fcbc28262', '0x734244C41DBa69bcCf52EE646aA9Fa105281002e', '0x0C90ad688fA5fBddC0195685631342DfE5113bf0', '0x9a4567cd19387D98cf1b19895490a83df53B8716', '0xFC592C5416B77F6c63fd847B6F78f9a6e4540729', '0x94a148276460fc80938021b7f6C8aFdc4c8E1175', '0x9d8510bB6b391044A761057879B56Fd91E9D994C', '0x13185335572e061F0eeAcB4237F519dBb9360dCA', '0x7f654D655dDa2c4897F03A3504be5644223a9ED5', '0x0F0C2fA686f8A86d6bd21b840ec190c84A12314c']
Balance: 1000 ETH


In [23]:
contract_source = """
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.0;

contract Crowdfunding {
    uint public campaignCount;

    struct Campaign {
        address creator;
        string title;
        uint goal;
        uint deadline;
        uint amountRaised;
        bool completed;
    }

    mapping(uint => Campaign) public campaigns;
    mapping(uint => mapping(address => uint)) public contributions;

    event CampaignCreated(uint campaignId, address creator, string title, uint goal);
    event Contributed(uint campaignId, address contributor, uint amount);
    event Completed(uint campaignId, uint amountRaised);

    function createCampaign(string memory title, uint goal, uint duration) public {
        campaignCount++;
        campaigns[campaignCount] = Campaign({
            creator: msg.sender,
            title: title,
            goal: goal,
            deadline: block.timestamp + duration,
            amountRaised: 0,
            completed: false
        });
        emit CampaignCreated(campaignCount, msg.sender, title, goal);
    }

    function contribute(uint campaignId) public payable {
        Campaign storage campaign = campaigns[campaignId];
        require(!campaign.completed, "Campaign completed");
        require(block.timestamp < campaign.deadline, "Campaign expired");

        campaign.amountRaised += msg.value;
        contributions[campaignId][msg.sender] += msg.value;
        emit Contributed(campaignId, msg.sender, msg.value);
    }

    function completeCampaign(uint campaignId) public {
        Campaign storage campaign = campaigns[campaignId];
        require(block.timestamp >= campaign.deadline, "Deadline not reached");
        campaign.completed = true;
        emit Completed(campaignId, campaign.amountRaised);
    }

    function getCampaignDetails(uint campaignId) public view returns (
        address creator,
        string memory title,
        uint goal,
        uint deadline,
        uint amountRaised,
        bool completed
    ) {
        Campaign memory campaign = campaigns[campaignId];
        return (
            campaign.creator,
            campaign.title,
            campaign.goal,
            campaign.deadline,
            campaign.amountRaised,
            campaign.completed
        );
    }
}
"""

In [24]:
# Compile contract
compiled = compile_source(contract_source)
contract_interface = compiled['<stdin>:Crowdfunding']


tx_hash = w3.eth.contract(
    abi=contract_interface['abi'],
    bytecode=contract_interface['bin']
).constructor().transact({
    'from': w3.eth.accounts[0],
    'gas': 3000000
})

tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
contract_address = tx_receipt.contractAddress

# Create contract instance
crowdfunding = w3.eth.contract(
    address=contract_address,
    abi=contract_interface['abi']
)

print(f" Contract successfully deployed at: {contract_address}")
print(f"Transaction receipt: {dict(tx_receipt)}")

✅ Contract successfully deployed at: 0xA5C54A27a142e7220D996633fd710a06bC8ccb11
Transaction receipt: {'transactionHash': HexBytes('0xb2f33bc1a581d5554f98583612e7045b851061738d0c9f4d90a758fbb07bebc6'), 'transactionIndex': 0, 'blockNumber': 1, 'blockHash': HexBytes('0x554a84617d8a4b1c07758913eabfa5f4ec7f7467ef204ec041a6c490ac2b9c34'), 'from': '0xa33CF0c4DeDa373483cBB60f59f7AD7Fcbc28262', 'to': None, 'cumulativeGasUsed': 910231, 'gasUsed': 910231, 'contractAddress': '0xA5C54A27a142e7220D996633fd710a06bC8ccb11', 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [25]:

print("\nVerifying deployment:")
print(f"Campaign count: {crowdfunding.functions.campaignCount().call()} (should be 0)")

# Testing- creating a campaign
tx_hash = crowdfunding.functions.createCampaign(
    "Test Campaign",
    w3.to_wei(1, 'ether'),
    300  # 5 min duration
).transact()
w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"\nCampaign count: {crowdfunding.functions.campaignCount().call()} (should be 1)")


details = crowdfunding.functions.getCampaignDetails(1).call()
print("\nCampaign details:")
print(f"Title: {details[1]}")
print(f"Goal: {w3.from_wei(details[2], 'ether')} ETH")
print(f"Deadline: {details[3]}")


Verifying deployment:
Campaign count: 0 (should be 0)

Campaign count: 1 (should be 1)

Campaign details:
Title: Test Campaign
Goal: 1 ETH
Deadline: 1754212135


**WEEK 8**

In [31]:
# Run this in Colab to generate a disposable test wallet
from web3 import Web3
import secrets

private_key = "0x" + secrets.token_hex(32)
account = Web3().eth.account.from_key(private_key)

print("🔐 TEST WALLET (FOR GOERLI ONLY)")
print(f"Address: {account.address}")
print(f"Private Key: {private_key}")

print("\n⚠️ WARNING: Never use this for real funds! Only for testing.")

🔐 TEST WALLET (FOR GOERLI ONLY)
Address: 0xb1503a23D17348fa4cAD4398c868158DbF358f85
Private Key: 0x6bc3d031406c56bcbf5ea86fcb6a30d9320ccff63a57b8a3bc507e52dd40886c

⚠️ WARNING: Never use this for real funds! Only for testing.


<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()


In [32]:
# Install required packages (run this first)
!pip install web3 py-solc-x python-dotenv
!python -m solcx.install v0.8.0

<frozen runpy>:128: RuntimeWarning: 'solcx.install' found in sys.modules after import of package 'solcx', but prior to execution of 'solcx.install'; this may result in unpredictable behaviour


In [40]:
from web3 import Web3
import secrets

# Generate a new test wallet (never use for mainnet!)
private_key = "0x" + secrets.token_hex(32)
account = Web3().eth.account.from_key(private_key)

print("🔐 TEST WALLET (GOERLI ONLY)")
print(f"Address: {account.address}")
print(f"Private Key: {private_key}")

print("\n⚠️ WARNING: Never use for real funds! Only for testing.")

🔐 TEST WALLET (GOERLI ONLY)
Address: 0x24e6D75c6a1C7994ab6F860D55d55b7F4F3AE6d2
Private Key: 0xb8c67f4a7a70a6100e86b75c74f5698a5d33c0a0aa6eb001be7fab17a5eabc1f

⚠️ WARNING: Never use for real funds! Only for testing.


<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()


In [41]:
# This code helps you sign up for a free key
from IPython.display import HTML
HTML('''
<a href="https://dashboard.alchemy.com/signup" target="_blank">
  <button style="background-color: #4CAF50; color: white; padding: 10px 20px; border: none; border-radius: 4px;">
    Get Free Alchemy API Key
  </button>
</a>
<p>After signing up:</p>
<ol>
  <li>Create new "Ethereum" app in dashboard</li>
  <li>Select "Goerli" network</li>
  <li>Copy API Key from "View Key" button</li>
</ol>
''')

In [43]:
from web3 import Web3
from solcx import compile_source

ALCHEMY_API_KEY = "https://eth-mainnet.g.alchemy.com/v2/1SgyFTcHX9839xUiLhNQ2"  # From Alchemy dashboard
PRIVATE_KEY = "0xb8c67f4a7a70a6100e86b75c74f5698a5d33c0a0aa6eb001be7fab17a5eabc1f"  # From generated wallet

w3 = Web3(Web3.HTTPProvider(f"https://eth-goerli.g.alchemy.com/v2/{ALCHEMY_API_KEY}"))
assert w3.is_connected(), "Failed to connect!"

# Verify balance
account = w3.eth.account.from_key(PRIVATE_KEY)
balance = w3.from_wei(w3.eth.get_balance(account.address), 'ether')
print(f"Balance: {balance} ETH")
if balance < 0.01:
    raise ValueError("Get test ETH from faucet first")

# Your contract code
contract_source = """[PASTE YOUR CONTRACT CODE HERE]"""

# Compile & deploy
compiled = compile_source(contract_source)
contract_interface = compiled['<stdin>:Crowdfunding']

tx = {
    'nonce': w3.eth.get_transaction_count(account.address),
    'gas': 3000000,
    'gasPrice': w3.to_wei('50', 'gwei'),
    'data': contract_interface['bin'],
    'chainId': 5  # Goerli
}

signed_tx = account.sign_transaction(tx)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(f"Contract deployed at: {receipt.contractAddress}")
print(f"🔗 Etherscan: https://goerli.etherscan.io/address/{receipt.contractAddress}")

AssertionError: Failed to connect!